# RLNN Tetris

*by Federico Larrieu and Tyson O'Leary, May 5, 2023*

GitHub: [https://github.com/falarrieu/rlnn-tetris](https://github.com/falarrieu/rlnn-tetris)

## Introduction

We both really enjoy Tetris and we are interested in applying reinforcement learning to a more complex problem than we've seen in class. Applying it to tetris will be constantly interesting, and the research we've done shows it also will not be trivial. 

For our project, we sought to create an AI to play Tetris. It would use a neural network and reinforcement learning to learn to play tetris optimally. To do so, would require us to either find an implementation of Tetris or implement the game ourselves, then define our states and actions to be used by the Q-net. We ended up just implementing the parts of the game logic that we needed to train the network we created.

The questions we sought to answer:
  * What is the highest number of cleared lines our AI can reach?
  * What is the highest score our AI can achieve?
  * Can we split this problem into multiple neural networks to solve different parts of the problem?

We hypothesize that copying the reinforcement learning methods we learned in class most likely will not produce satisfactory results. We plan to explore multiple definitions for the actions and states, as well as multiple architectures like an ensemble model. We will try different implementations of our neural network including the classes from the lecture notes as well as other implementations from the pytorch documentation. 

After experimenting with different implementations, we found the one adapted from pytorch to work best. We then performed a grid search on our hyperparameters to find the best architecture, with the goal to have at least 75% accuracy.

## Methods

Our approach to creating an AI for Tetris involves breaking down the problem into subproblems. The first subproblem is determining the best placement for a piece, while the second subproblem involves moving the piece into the desired position and rotation. If time permits, we may also incorporate a piece selection strategy. We planned to use two reinforced neural networks, each responsible for one subproblem, and deploy them in an ensemble model architecture.

First, the model will input a still frame of the game. If a piece is being held, both the held piece and the current piece will be run through the position neural network, and the position with the better placement will be output. If no piece is being held, then the next piece and the current piece will be run through the position neural network, and again the position with the better placement will be output. The best position will then be fed into the movement neural network, which will determine the next best action to take to move towards that position.

<img src="EnsembleModel.drawio.svg" alt= “EnsembleModel” width="500" height="600">
<p style="text-align: center;"><em>Figure 1: Ensemble Architecture</em></p>

### Position Reinforced Neural Network

<img src="PositionRLNN.drawio.svg" alt= “PositionRLNN” width="400" height="500">
<p style="text-align: center;"><em>Figure 2: Position Agent I/O</em></p>

The first agent in the ensemble architecture in the position agent. This agent is responsible for deciding the best position to place a piece on the board. It will be used at the introduction of each current piece and the held or next piece. The optimal piece will be chosen and placed on the board. The input of the neural network is quite simple in the fact that it will only take the current board state and a piece. For the output we are looking for the best position and rotation of the piece.

<img src="PositionTraining.drawio.svg" alt= “PositionTraining” width="400" height="500">
<p style="text-align: center;"><em>Figure 3: Position Agent Training</em></p>

To train this network we planned to create a environment loop in which the agent will place pieces and collect rewards. The initial board environment will be empty to allow for intital testing of the model. We also planned to introduced randomnly or procedurally generated boards to add noise and better generalize the model to handle a multitude of board states.

### Movement Reinforced Neural Network

<img src="MovementRLNN.drawio.svg" alt= “MovementRLNN” width="400" height="500">
<p style="text-align: center;"><em>Figure 4: Movement Agent I/O</em></p>

The second agent in our ensemble architecture is the movement agent. It has the single responsibility of moving a piece into a specified position and rotation. The input of this agent is similar to the position agent as it includes the board state and the current piece. It differes in the sense that it also required the destination position and rotation, and also all the possbile actions it can take. In Tetris the possible actions are shift left, shift right, rotate counter clockwise, rotate clockwise and drop. Actions were filtered to only include possible actions for the current frame. The output of the agent is the action that we want to apply.

<img src="MovementTraining.drawio.svg" alt= “MovementTraining” width="400" height="500">
<p style="text-align: center;"><em>Figure 5: Movement Agent Training</em></p>

The environment for training this agent is also similar to that of the position agent. The initial board environment will be empty to allow for intital testing of the model. We also planned to introduced randomnly or procedurally generated boards to add noise and better generalize the model to handle a multitude of board states. This is especially important for the movement agent for it needs to be able to maneuver around obstacles if need be.

### Process
We began by adapting code from the lecture notes on reinforcement learning to build the movement neural network architecture. Building the optimizer and neural network classes was as simple as a copy-paste, with some small changes for our use case. The next step was to write our `initial_state`, `next_state`, `reinforcement`, `epsilon_greedy`, and `make_samples` functions to support reinforcement learning. We chose to have each sample be a single movement of the Tetris piece, meaning one action would be chosen. Each trial would be one piece moving from the top of the board to a locking position at the bottom, so each trial would have at most 20 samples because the board is 20 pixels tall. This naturally lead to us defining the initial state to be a new piece placed in the top center of the board and next state being the current piece moved however the epsilon greedy policy chooses. We defined our epsilon greedy policy similarly to how it was done for the marble example in lecture, where it would either randomly pick an action or choose the best action according to the neural network. We improved it by restricting the possible actions that could be chosen based on whether the move would be valid. For example, if any squares on the piece are touching the right wall of the board, the piece should not be able to move to the right. With all of the logic defined, we could then define our reinforcement policy as follows:

1. If the piece gets closer to its goal in the x direction, increment reinforcement by 1. Otherwise decrement by 1.
2. If the piece gets closer to its goal in the y direction, increment reinforcement by 1. Otherwise decrement by 1.
3. If the piece gets closer to the correct rotation, increment reinforcement by 1. Otherwise decrement by 1.
4. If the piece reaches its goal exactly, increment reinforcement by 20.
5. The total of all of these increments and decrements is the reinforcement for the current action.

Completing the reinforcements meant we were done defining our policy. As a part of this process we also wrote multiple classes for controlling the game logic. 

* A board class to store the board state and provide logic for moving pieces within the board
* A Game class for controlling next piece choice and game rules
* A class for each shape of piece that all inherit from a Tetromino class for defining individual piece types

We then started training the model, with very little success. No matter how much we trained, our loss would not decrease over the training time. We would get decreases in loss for each set of samples, but it would not improve overall. To try to find improvement, we started by reducing the state space. We stopped the pieces from locking in place, which meant that a piece would always be trying to reach a position at the bottom of the board. Next, we tried using only one type of piece for training, and then always having the same goal. Our final attempt was giving the piece multiple tries per trial to reach the goal to see if having more samples in one trial would allow for better training. Much to our dismay, none of this made any difference in our accuracy. 

After failing to fix our current implementation, we pivoted to using a Deep Q Learning network implemented with pytorch, which we adapted from [this tutorial](https://pytorch.org/tutorials/intermediate/reinforcement_q_learning.html). It was a joy working with pytorch for this implementation, and we learned a lot about techniques for reinforcement learning that weren't touched on in lecture. To begin, we generalized their DQN module to allow us to pass an architecture for the fully connected layers on construction. One big difference between this implementation and our original one is that the training for the DQN happens outside of the neural network class. It was difficult to change the way we were thinking about our code. Once we got the code working, this instantly improved our results. The agent would actually learn to move the pieces, and with enough training we were able to get very high accuracy.

We believe there was some problem with the way we adapted the network from the lecture notes that was causing our bad results. However, the DQN had some other optimizations that seemed to contribute to our success. The DQN includes a replay memory, which means the network doesn't forget samples from previous trials, allowing it to continue learning from older attempts. The replay memory has a set size, which means it will begin to forget trials as they become too old, so it's always improving. It also uses a policy network and a target network. The idea behind the target network is to decouple the Q-value estimates from the network  being updated, which helps to stabilize training the network.

### Movement Neural Network Hyperparameter Optimization

Switching to a DQN has introduced new hyperparameters that we have not learned or used in class. This includes batch size and tau. We approached hyperparameter optimization by first individually optimizing each hyperparameter whilst maintaining the other hyperparameters constant. This approach was to understand that each new hyperparameter would affect accuracy of a DQN model. We had the same approach with the hyperparameters that we had learned before. This was to verify that using a DQN model will not act differently then what we have seen before.

Using what we gained from individually optimizing and understanding the trends of each hyperparameter, we ran a simple grid search without cross validation. This was done simply with the use of several nested for loops in which each would iterate through a list of different values for the respective hyperparameter.

#### Batch

Batch size helps determine the number of transitions that will be sampled from the replay memory in order to optimize the model. The larger the batch size the more stable our model will perform. Smaller batch sizes introduce more noise into the model thus leading into instability. Transitions which are held in the replay memory buffer are composed of the current state, action, next state, and a tuple of rewards. Each of the composed components are then transposed into batch tensors. With these tensors we can then compute the Q values of the state and actions within our transitions. We then compute the loss, minimize the loss using the chosen optimizer (Adam or SGD), and update the parameters of the neural network.

![OptBatch.png](OptBatch.png)
<p style="text-align: center;"><em>Figure 6: Batch Size Optimization</em></p>


   To optimize for the batch size, we researched common ranges of batch sizes and decided to test using batch sizes of 32, 64, and 128. For each batch size, we kept the other hyperparameters constant, including GAMMA = 0.25, EPS_START = 0.9, EPS_END = 0.05, EPS_DECAY = 1000, TAU = 0.005, LR = 1e-4, n_hiddens_per_layer=[512, 256, 128], num_episodes = 500, and optimizer="adam". In our results (Fig 6.) we observe that a batch size of 64 achieved a final average test accuracy of %35 and and batch size of 128 achieved a final average test accuracy of %37. Being that the accuracy of 64 and 128 batch sizes are quite similar, we might want to choose a batch size 64 to save on computation.

#### Gamma

Gamma is the discount factor that is used in the Bellman equation. Gamma is used to weigh the importance of future rewards relevant to immediate rewards. The closer the value is to 0 means we are wanting to weigh immediate rewards larger. If the value is closer to 1 than we are placing more emphasis on future rewards.

![OptGamma.png](OptGamma.png)
<p style="text-align: center;"><em>Figure 7: Gamma Optimization</em></p>

   To optimize for the gamma, we tested using gamma values of 0.1, 0.25, 0.5, 0.75 and 0.9. For each value, we kept the other hyperparameters constant, including BATCH_SIZE= 128, EPS_START = 0.9, EPS_END = 0.05, EPS_DECAY = 1000, TAU = 0.005, LR = 1e-4, n_hiddens_per_layer=[512, 256, 128], num_episodes = 500, and optimizer="adam". In our result we observed the best final accuracy of %35 with a gamma value of 0.25.

#### Tau


Tau determines the rate at which the target network's weights are updated with the weights of the policy network. In other words, the target network's weights are updated by interpolating with the policy network's weights with a rate of Tau. The purpose of doing this is to provide a more stable set of targets for the Q-value function. The higher the Tau value, the slower the update rate of the target network, and vice versa.

![OptTau.png](OptTau.png)
<p style="text-align: center;"><em>Figure 8: Tau Optimization</em></p>

To optimize for the tau, we tested using values of 0.001, 0.005, 0.01, 0.05 and 0.1. For each value, we kept the other hyperparameters constant, including BATCH_SIZE= 128, GAMMA=0.25, EPS_START = 0.9, EPS_END = 0.05, EPS_DECAY = 1000, LR = 1e-4, n_hiddens_per_layer=[512, 256, 128], num_episodes = 500, and optimizer="adam". In our result we observed the best accuracy %37 with a tau value of 0.01.


#### Learning Rate

The learning rate is a hyperparameter that determines the step size at each iteration while moving toward a minimum of a loss function during training of a neural network.

![OptLearningRate.png](OptLearningRate.png)
<p style="text-align: center;"><em>Figure 9: Learning Rate Optimization</em></p>

To optimize for the learning rate, we tested using values of 0.0001, 0.001, 0.01 and 0.1. For each value, we kept the other hyperparameters constant, including BATCH_SIZE= 128, GAMMA=0.25, EPS_START = 0.9, EPS_END = 0.05, EPS_DECAY = 1000, TAU = 0.005, n_hiddens_per_layer=[512, 256, 128], num_episodes = 500, and optimizer="adam". In our result we observed the best accuracy %38 with a learning rate value of 0.001.

#### N Hiddens Per Layer

The number of hidden units per layer is a hyperparameter that determines the architecture of the neural network. A larger number of hidden units can potentially improve the model's ability to capture patterns in the data, but it can also increase overfitting.

![OptArch.png](OptArch.png)
<p style="text-align: center;"><em>Figure 10: Architecture Optimization</em></p>

To optimize for the number of hidden layers, we tested using different architectures of [50, 25], [512, 256, 128], [1024, 512, 256, 128]. For each architecture, we kept the other hyperparameters constant, including BATCH_SIZE= 128, GAMMA=0.25, EPS_START = 0.9, EPS_END = 0.05, EPS_DECAY = 1000, TAU = 0.005,  LR = 1e-4, num_episodes = 500, and optimizer="adam". In our result we observed the best accuracy was achieved from the largest network architecture. Although, the second best network architecture was close in accuracy and may be a better choice to save on computation.

#### Episodes

Episodes are in essence the number of times we allow the agent to interact with the tetris environment. Specifically in context of the movement agent, an episode was moving a piece from the to finish or failure. 

![OptEpisodes.png](OptEpisodes.png)
<p style="text-align: center;"><em>Figure 11: Episode Optimization</em></p>

To optimize for the number of episodes, we tested using the values 10, 1000, 5000 and 10000. For each value, we kept the other hyperparameters constant, including BATCH_SIZE= 128, GAMMA=0.25, EPS_START = 0.9, EPS_END = 0.05, EPS_DECAY = 1000, TAU = 0.005, LR = 1e-4, n_hiddens_per_layer=[512, 256, 128], and optimizer="adam". In our result we observed the the 1000, 5000 and 10000 episodes seem to be quite close in accuracy. In our final model we experiment with all three values.

## Results



![GridSearchDF.png](GridSearchDF.png)
<p style="text-align: center;"><em>Figure 12: Grid Search</em></p>

Above is the dataframe showing the results from our grid search. This one is not sorted, but the optimal architecture was 

* Batch Size: 64
* Gamma: 0.25
* Tau: 0.01
* Learning Rate: 0.001
* Architecture: [1024, 512, 256 128]

We found it interesting that a lower gamma value worked better. This makes some sense because it doesn't take very many actions to reach the bottom of a tetris board, and most actions taken can be greedy. The large architecture makes sense as well, because of the huge state space created by the board and many types of pieces. It's possible that the network could be improved even more if we used an even larger network. This one was the largest we tried.


![OptimalModel5000.png](OptimalModel5000.png)
<p style="text-align: center;"><em>Figure 13: Optimal Model</em></p>

In the figure above (Fig 13) we have trained the model with the optimized hyperparameters for 5000 episodes. On the left chart, which charts the duration of each episode, we observe that the average episode takes around ~17.5 actions. Ideally we would have liked to see the average duration to be less. This would mean the this model would have chosen the drop action more frequently. The right chart, which plots are Hubert loss, we are achieving a decent loss. With a much larger number of episodes we may be able to closer approach 0.

![OptimalModelAccuracy5000.png](OptimalModelAccuracy5000.png)
<p style="text-align: center;"><em>Figure 13: Optimal Model Accuracy 5000 Episodes</em></p>

![OptimalModelAccuracy10000.png](OptimalModelAccuracy10000.png)
<p style="text-align: center;"><em>Figure 13: Optimal Model Accuracy 10000 Episodes</em></p>

![Animation.gif](Animation.gif)
<p style="text-align: center;"><em>Figure 14: Optimal Model Tests</em></p>

The model trained on 5000 episodes reached an average accuracy of 75%, while the model trianed on 10000 episodes reach an average accuracy of 78%. This tells us that the model is continuing to improve with more episodes, but we do see diminishing returns. If we wanted ideal accuracy, we would have to train the model for many, many hours. However, we did reach our goal of having an accuracy above 75%, which we are very proud of. The GIF shows the 10000 episode model moving pieces to their goals. It is on target or slightly off target every time. It's a very satisfying result!

## Conclusions

In working on this project, we gained valuable knowledge and further solidified our understanding of machine learning techniques. During the inception phase, we quickly reviewed various implementations of the Tetris problem, which allowed us to observe the flexibility of problem-solving in machine learning. Each paper had similar but different approaches to tackling the Tetris problem, giving us the ability to be more creative in composing our solution.

Our hypothesis was that decomposing the Tetris problem into subproblems would enable us to assign responsibility to different agents, each finely tuned to handle one problem. However, when implementing the agent responsible for moving the tetrominoes (MovementNN), we initially encountered issues with the agent not learning patterns, even though there were no obvious errors in implementation. We suspected the large game state space of Tetris might be causing the problem, but after testing the agent through 100,000s of trials, we found no improvement.
With the due date fast approaching, we decided to proceed with using the Deep Q Learning methodology. We re-implemented the neural network, incorporated replay memory, and added a separate trainer implementation. Our efforts yielded positive results with near 50% accuracy in testing, without hyperparameter optimization and under 1000 episodes. After, we began to conduct hyperparameter optimization as well as increasing the number of episodes. 

Optmizing the hyperparameters and increasing the number of episodes drastically improved the performance of the movement agent. After running the optimal model with 10000 episodes (Fig 13) we achieved a 78% average accuracy.

Moving forward, we would like to improve the performance of the model by training it with a larger number of episodes. We would also like to re-iterate on the reinforcements to try and motivate the model to place the piece quicker. Lastly, we would like to implement the procedural and/or random generation for adding noise th


## Contributions

**Tyson O'Leary**
- Tetrominos
- Piece Provider 
- Valid Placement Generator
- Board
- Game
- MovementNN
- Neural Network 
- Neural Network 2
- NNTrainer
- Vector
- Presentation
- Report

**Federico Larrieu**
- Client
- Board
- Game
- MovementNN
- Neural Network
- Neural Network 2
- NNTrainer
- Hyperparameter Optimization
- Presentation
- Report


### References

* [Goodfellow, et al., 2016] Ian Goodfellow and Yoshua Bengio and Aaron Courville, [Deep Learning](http://www.deeplearningbook.org), MIT Press. 2016.

Your report for a single person team should contain **approximately** 2,000 words times number of team members, in markdown cells.  You can count words by running the following python code in your report directory.  Projects with two people, for example, should contain about 4,000 words, a four-person team should submit a report of approximately 8,000 words.

Of course, your results and analysis speak much more than the word count.  Deep analysis in a shorter form is better than vague, over-wordy non-analysis.

In [1]:
import io
from nbformat import current
import glob
nbfile = glob.glob('FinalReport.ipynb')
if len(nbfile) > 1:
    print('More than one ipynb file. Using the first one.  nbfile=', nbfile)
with io.open(nbfile[0], 'r', encoding='utf-8') as f:
    nb = current.read(f, 'json')
word_count = 0
for cell in nb.worksheets[0].cells:
    if cell.cell_type == "markdown":
        word_count += len(cell['source'].replace('#', '').lstrip().split(' '))
print('Word count for file', nbfile[0], 'is', word_count)

Word count for file FinalReport.ipynb is 3488


/home/tmoleary/.local/lib/python3.8/site-packages/nbformat/current.py:15: UserWarning: nbformat.current is deprecated.

- use nbformat for read/write/validate public API
- use nbformat.vX directly to composing notebooks of a particular version

  warnings.warn("""nbformat.current is deprecated.
